## 모멘텀 현상
군집 행동, 정박 효과, 확증 편향, 처분 효과 등의 행동 편향에 의해서 발생

편향들로 인한 정보에 대한 과소/과대평가가 가격의 비효율성으로 이어져 투자자의 비이성적 행동을 초래


---------

상대 강도 : 26주 이동평균선을 기준으로 더 많이 오른 종목

상대적 모멘텀 : 최근 6~12개월 동안 상대적으로 수익률이 높은 종목을 매수하는 전략

절대적 모멘텀 : 상승장에서만 투자하고 하락장에서는 미국 단기 국채나 현금으로 갈아타는 전략

## 듀얼 모메텀 투자
상대적 모멘텀 전략과, 절대적 모멘텀 전략을 하나로 합친 듀얼 전략



In [12]:
import pandas as pd
import pymysql
from datetime import datetime
from datetime import timedelta
from Investar import Analyzer

class DualMomentum:
    def __init__(self):
        """생성자: KRX 종목코드(codes)를 구하기 위한 MarkgetDB 객체 생성"""
        self.mk = Analyzer.MarketDB()
    
    def get_rltv_momentum(self, start_date, end_date, stock_count):
        """특정 기간 동안 수익률이 제일 높았던 stock_count 개의 종목들 (상대 모멘텀)
            - start_date  : 상대 모멘텀을 구할 시작일자 ('2020-01-01')   
            - end_date    : 상대 모멘텀을 구할 종료일자 ('2020-12-31')
            - stock_count : 상대 모멘텀을 구할 종목수
        """       
        connection = pymysql.connect(host='localhost', port=3307, 
            db='INVESTAR', user='root', passwd='angel1324', autocommit=True)
        cursor = connection.cursor()
        
        # 사용자가 입력한 시작일자를 DB에서 조회되는 일자로 보정 
        sql = f"select max(date) from daily_price where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("start_date : {} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')


        # 사용자가 입력한 종료일자를 DB에서 조회되는 일자로 보정
        sql = f"select max(date) from daily_price where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("end_date : {} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')


        # KRX 종목별 수익률을 구해서 2차원 리스트 형태로 추가
        rows = []
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(self.mk.codes):            
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price / old_price - 1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price, 
                returns])


        # 상대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns', ascending=False)
        df = df.head(stock_count)
        df.index = pd.Index(range(stock_count))
        connection.close()
        print(df)
        print(f"\nRelative momentum ({start_date} ~ {end_date}) : "\
            f"{df['returns'].mean():.2f}% \n")
        return df
    
    def get_abs_momentum(self, rltv_momentum, start_date, end_date):
        """특정 기간 동안 상대 모멘텀에 투자했을 때의 평균 수익률 (절대 모멘텀)
            - rltv_momentum : get_rltv_momentum() 함수의 리턴값 (상대 모멘텀)
            - start_date    : 절대 모멘텀을 구할 매수일 ('2020-01-01')   
            - end_date      : 절대 모멘텀을 구할 매도일 ('2020-12-31')
        """
        stockList = list(rltv_momentum['code'])        
        connection = pymysql.connect(host='localhost', port=3307, 
            db='INVESTAR', user='root', passwd='angel1324', autocommit=True)
        cursor = connection.cursor()


        # 사용자가 입력한 매수일을 DB에서 조회되는 일자로 변경 
        sql = f"select max(date) from daily_price "\
            f"where date <= '{start_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("{} -> returned None".format(sql))
            return
        start_date = result[0].strftime('%Y-%m-%d')


        # 사용자가 입력한 매도일을 DB에서 조회되는 일자로 변경 
        sql = f"select max(date) from daily_price "\
            f"where date <= '{end_date}'"
        cursor.execute(sql)
        result = cursor.fetchone()
        if (result[0] is None):
            print ("{} -> returned None".format(sql))
            return
        end_date = result[0].strftime('%Y-%m-%d')


        # 상대 모멘텀의 종목별 수익률을 구해서 2차원 리스트 형태로 추가
        rows = []
        columns = ['code', 'company', 'old_price', 'new_price', 'returns']
        for _, code in enumerate(stockList):            
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{start_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            old_price = int(result[0])
            sql = f"select close from daily_price "\
                f"where code='{code}' and date='{end_date}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            if (result is None):
                continue
            new_price = int(result[0])
            returns = (new_price / old_price - 1) * 100
            rows.append([code, self.mk.codes[code], old_price, new_price,
                returns])


        # 절대 모멘텀 데이터프레임을 생성한 후 수익률순으로 출력
        df = pd.DataFrame(rows, columns=columns)
        df = df[['code', 'company', 'old_price', 'new_price', 'returns']]
        df = df.sort_values(by='returns', ascending=False)
        connection.close()
        print(df)
        print(f"\nAbasolute momentum ({start_date} ~ {end_date}) : "\
            f"{df['returns'].mean():.2f}%")
        return


## 상대 모멘텀
특정 기간 동안 상대적으로 수익률이 좋았던 n개의 항목을 구함

In [13]:
dm = DualMomentum()
rm = dm.get_rltv_momentum('2020-07-01', '2020-12-31', 300)

       code   company  old_price  new_price      returns
0    032860       휴먼엔        195       9780  4915.384615
1    025560      미래산업         79       3040  3748.101266
2    223310  경남제약헬스케어        166       3320  1900.000000
3    054220     비츠로시스        485       5930  1122.680412
4    206400     베노홀딩스        799       7940   893.742178
..      ...       ...        ...        ...          ...
295  092870       엑시콘       9200      16500    79.347826
296  020180    대신정보통신       1085       1945    79.262673
297  109820     진매트릭스      14500      25950    78.965517
298  000990     DB하이텍      28500      51000    78.947368
299  041190    우리기술투자       2725       4870    78.715596

[300 rows x 5 columns]

Relative momentum (2020-07-01 ~ 2020-12-30) : 197.14% 



## 절대 모멘텀
자산의 가치가 상승하고 있을 때만 투자

In [15]:
am = dm.get_abs_momentum(rm, '2021-01-01', '2021-06-31')

       code  company  old_price  new_price     returns
1    025560     미래산업       3040      18950  523.355263
279  224020  에스케이씨에스        680       3250  377.941176
293  298020    효성티앤씨     211000     900000  326.540284
157  181340    이즈미디어       6690      25100  275.186846
62   011200      HMM      13950      43900  214.695341
..      ...      ...        ...        ...         ...
27   112610    씨에스윈드     177000      77500  -56.214689
19   270660     에브리봇      98800      42300  -57.186235
51   012600  센트럴인사이트       9200       2490  -72.934783
186  033540      파라텍      13500       3260  -75.851852
102  260970     에스앤디     300000      36450  -87.850000

[300 rows x 5 columns]

Abasolute momentum (2020-12-30 ~ 2021-06-30) : 14.97%


## 한국형 듀얼 모멘텀
한국 시장은 3개월 전략이 12개월 전략보다 효과적

In [16]:
dm = DualMomentum()
rm = dm.get_rltv_momentum('2021-01-01', '2021-03-31', 10)

     code  company  old_price  new_price      returns
0  052190    세영디앤씨        295       8850  2900.000000
1  066980  이엔코퍼레이션        720      11600  1511.111111
2  058530   슈펙스비앤피        162       1620   900.000000
3  050120     ES큐브        558       5120   817.562724
4  194480   데브시스터즈      14450     115000   695.847751
5  196450  코아시아옵틱스        411       2055   400.000000
6  121850      코이즈       1405       6980   396.797153
7  053290     NE능률       2845      12000   321.792619
8  058420     제이웨이        546       2160   295.604396
9  020560   아시아나항공       4210      15150   259.857482

Relative momentum (2020-12-30 ~ 2021-03-31) : 849.86% 



In [17]:
am = dm.get_abs_momentum(rm, '2021-03-31', '2021-06-31')

     code  company  old_price  new_price    returns
7  053290     NE능률      12000      22400  86.666667
3  050120     ES큐브       5120       6590  28.710938
5  196450  코아시아옵틱스       2055       2350  14.355231
9  020560   아시아나항공      15150      17200  13.531353
0  052190    세영디앤씨       8850       8850   0.000000
2  058530   슈펙스비앤피       1620       1620   0.000000
8  058420     제이웨이       2160       2160   0.000000
1  066980  이엔코퍼레이션      11600      11350  -2.155172
4  194480   데브시스터즈     115000      91600 -20.347826
6  121850      코이즈       6980       3245 -53.510029

Abasolute momentum (2021-03-31 ~ 2021-06-30) : 6.73%
